In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import shap
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
engagement=pd.read_csv(r'C:\A\Education\Data Science\Relax Inc\takehome_user_engagement.csv')
user=pd.read_csv(r'C:\A\Education\Data Science\Relax Inc\takehome_users.csv',encoding='latin-1')

In [3]:
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
# it seems there is no missing value in this dataset
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,22-04-2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,15-11-2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,19-03-2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,21-05-2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,17-01-2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [6]:
# there is missing value in 'last_session_creation_time' and 'invited_by_user_id'
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [7]:
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp']).dt.date
engagement.head(30)

,time_stamp,user_id,visited
0,2014-04-22,1,1
1,2013-11-15,2,1
2,2013-11-29,2,1
3,2013-12-09,2,1
4,2013-12-25,2,1
5,2013-12-31,2,1
6,2014-01-08,2,1
7,2014-02-03,2,1
8,2014-02-08,2,1
9,2014-02-09,2,1


In [8]:
engagement['adopted']=0

In [10]:
for i in range(len(engagement)-2):
    time=engagement['time_stamp'].iloc[i+2]-engagement['time_stamp'].iloc[i]
    if engagement['user_id'].iloc[i]==engagement['user_id'].iloc[i+1]==engagement['user_id'].iloc[i+2] and time.days<=7:
        engagement['adopted'].iloc[i]=1

In [12]:
engagement.head(100)

,time_stamp,user_id,visited,adopted
0,2014-04-22,1,1,0
1,2013-11-15,2,1,0
2,2013-11-29,2,1,0
3,2013-12-09,2,1,0
4,2013-12-25,2,1,0
5,2013-12-31,2,1,0
6,2014-01-08,2,1,0
7,2014-02-03,2,1,1
8,2014-02-08,2,1,1
9,2014-02-09,2,1,1


In [14]:
# pickle engagement
outfile = open(r'C:\A\Education\Data Science\Relax Inc\adopted_user','wb')
pickle.dump(engagement,outfile)
outfile.close()